In [79]:
import numpy as np
import pandas as pd
import pickle, csv
import os

from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Dropout, Input, concatenate, Flatten, Reshape
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow.keras.backend as K
from skimage.transform import resize

from matplotlib import pyplot as plt
from matplotlib import style
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns

In [80]:
# Load powerprice data and format it
df_powerprice = pd.read_csv("data/PowerPrice.csv")
df_powerprice["Time"] = pd.to_datetime(df_powerprice["Time"], infer_datetime_format=True)
df_powerprice = df_powerprice.drop(columns=["Time"])

In [81]:
# Load spatial data and format it
datasets = []
for filename in os.listdir("data/Spatial"):
    if filename.endswith(".pickle"):
        with open(os.path.join("data/Spatial", filename), "rb") as file:
            data = pickle.load(file)
            if data.shape == (100, 35040):
                datasets.append(data)
            else:
                print(f"Skipping {filename}: unexpected shape {data.shape}")
data = np.concatenate(datasets, axis=0)
data = data.T
df_spatial = pd.DataFrame(data)

In [82]:
df_spatial.shape

(35040, 1300)

In [83]:
#load the mask.csv
mask_df = pd.read_csv("germany/mask.csv", header=None)
mask_df.shape

(40, 40)

In [91]:
# Reshape mask_df to match the desired output shape
mask = mask_df.values.reshape((1, 40, 40, 1))
mask.shape

(1, 40, 40, 1)

In [100]:
n_samples = df_spatial.shape[0]
spatial = df_spatial.values.reshape((n_samples, 1, 1, 1300))
spatial = np.tile(spatial, (1, 40, 40, 1))

MemoryError: Unable to allocate 543. GiB for an array with shape (56064000, 1300) and data type float64

In [74]:
input1 = Input(shape=(35040, 1300, 1))
input2 = Input(shape=(40, 40, 1))

In [ ]:
# Apply 2D convolution on input2
x = Conv2D(filters=16, kernel_size=3, activation='relu')(input2)
x = GlobalMaxPooling2D()(x)

# Reshape input1 into a 3D tensor
y = Reshape(target_shape=(35040, 1300, 1))(input1)

# Concatenate output of 2D convolution with flattened 3D tensor
y = Concatenate()([Flatten()(y), x])

# Define output layer
output = Dense(units=10, activation='softmax')(y)

# Define the model
model = Model(inputs=[input1, input2], outputs=output)